### 1. 패키지 설치

### 2. 문서 split 및 Chroma를 활용한 vector store 구성

In [1]:
from huggingface_hub import whoami

try:
    user_info = whoami()
    print(f"로그인 상태입니다. 사용자: {user_info['name']}")
except Exception as e:
    print("로그인되지 않았거나 토큰이 유효하지 않습니다.")
    print(e)

/home/chae/faiss_env/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


로그인 상태입니다. 사용자: chaeeee


In [17]:
from langchain_community.document_loaders import Docx2txtLoader
from langchain.text_splitter import CharacterTextSplitter

from langchain.schema import Document

# 텍스트 분할 설정
text_splitter = CharacterTextSplitter(
    chunk_size=800,
    chunk_overlap=128  # 오버랩 설정
)

# 단일 DOCX 파일 로드
file_path = "./dataset2.docx"  # 파일 경로를 이곳에 입력하세요
loader = Docx2txtLoader(file_path)
raw_text = loader.load()[0].page_content  # DOCX 파일의 전체 텍스트 가져오기
print("raw_text 개수: ", len(raw_text))
      
# 작품명을 기준으로 텍스트 분리
def split_artwork_documents(doc_text):
    artworks = doc_text.split("\n\n작품명:")  # 작품을 구분
    documents = []

    for artwork in artworks:
        if artwork.strip():  # 빈 텍스트 제외
            # "작품명:" 추가로 일관성 유지
            doc_content = "작품명:" + artwork if not artwork.startswith("작품명:") else artwork

            # 메타데이터 초기화
            metadata = {}
            lines = doc_content.split("\n")  # 텍스트 줄 단위로 나누기

            # 메타데이터 추출
            for line in lines:
                if line.startswith("작품명:"):
                    metadata["작품명"] = line.replace("작품명:", "").strip()
                elif line.startswith("작가:"):
                    metadata["작가"] = line.replace("작가:", "").strip()
                elif line.startswith("제작 연도:"):
                    metadata["제작 연도"] = line.replace("제작 연도:", "").strip()
                elif line.startswith("카테고리:"):
                    metadata["카테고리"] = line.replace("카테고리:", "").strip()

            # Document 객체 생성
            documents.append(Document(
                page_content=doc_content.strip(),
                metadata=metadata
            ))

    return documents



# 작품별 Document 생성
documents = split_artwork_documents(raw_text)

# 생성된 작품별 Document에 대해 추가 청크 분할
chunked_documents = []
for doc in documents:
    chunks = text_splitter.split_text(doc.page_content)
    for chunk in chunks:
        # 청크에 원래 Document의 메타데이터 유지
        chunked_documents.append(Document(
            page_content=chunk,
            metadata=doc.metadata  # 원본 메타데이터 복사
        ))

print(f"총 문서 수: {len(chunked_documents)}")

raw_text 개수:  8042447
총 문서 수: 15195


In [18]:
# 첫 2개 문서만 출력해 확인
for i, doc in enumerate(chunked_documents[:2], 1):
    print(f"Document {i}:")
    print("메타데이터:", doc.metadata)
    print(doc.page_content)
    print("-" * 50)

Document 1:
메타데이터: {'작품명': '작가사진 / 作家寫眞 / Selfportrait', '작가': '한기석 / HAN Kisuk', '제작 연도': '1960', '카테고리': '사진'}
작품명: 작가사진 / 作家寫眞 / Selfportrait

작가: 한기석 / HAN Kisuk

작품 번호: 1

제작 연도: 1960

크기: 41×51

재료: 종이에 젤라틴실버프린트

카테고리: 사진

작품 설명: ‘농(Nong)’이라는 이름으로 미국에서 널리 알려진 한농(韓農) 한기석(1930-2011)은 국내 활동이 그리 많지 않아서 한국 화단에서는 생소한 이름이다. 그가 최초로 한국 화단에 등장한 것은 1971년 11월 신세계 화랑에서 개최한《Nong 展》이후이다. 그는 농(Nong)을 구름 위의 시선(詩仙) 혹은 주선(酒仙)같은 존재로 비유해서 미국에서 자신의 이름으로 쓰고 있다.그의 작품은 전반적으로 자신의 철학적 이미지를 조형화시킨 추상 회화 계통이다. 일종의 형이상학적인 회화 혹은 초현실적인 환상세계라고도 할 수 있는 그의 작품은 양식적인 면에서 주로 구상적인 형태를 취한다.한기석의 <작가사진>(1960)은 본인의 얼굴을 찍은 것으로, 사진 속에서 작가는 자신의 작품을 배경으로 화면의 우측을 주시하고 있다.
--------------------------------------------------
Document 2:
메타데이터: {'작품명': '팔괘호 / 八卦壺 / Palgwae Vase', '작가': '한기석 / HAN Kisuk', '제작 연도': '1960', '카테고리': '회화 II'}
작품명: 팔괘호 / 八卦壺 / Palgwae Vase

작가: 한기석 / HAN Kisuk

작품 번호: 2

제작 연도: 1960

크기: 250×127

재료: 캔버스, 종이에 유화 물감

카테고리: 회화 II

작품 설명: 한농(韓農) 한기석(1930-2011)은 표면 묘사에 많은 관심을 가진 작가이다.그는 모나고 약간 무게가 있는 듯이 보이는 

In [19]:
documents[11000]

Document(metadata={'작품명': '직각 / N/A / Right Angle', '작가': '김차섭 / KIM Tchahsup', '제작 연도': '1981', '카테고리': '판화'}, page_content='작품명: 직각 / N/A / Right Angle\n\n작가: 김차섭 / KIM Tchahsup\n\n작품 번호: 11001\n\n제작 연도: 1981\n\n크기: 57×76.5\n\n재료: 종이에 에칭\n\n카테고리: 판화\n\n작품 설명: 김차섭(金次燮, 1942-2022)은 일본 야마구치현(山口県)에서 태어났으며, 광복 즈음에 한국으로 건너와 1963년 서울대학교 회화과에 입학했다. 1960년대 후반에 《회화 ‘68》전과 《AG》전에 참여했으며, 1967년 제5회 파리비엔날레, 1970년 제7회 도쿄 국제판화비엔날레, 1971년 제11회 상파울루비엔날레 등에 참가하며 국제적으로도 활발히 활동하였다. 김차섭은 록펠러 재단 장학금을 받고 1975년에 미국으로 건너가서 뉴욕 프랫 인스티튜트(Pratt Institute) 대학원에서 에칭을 공부했다. 1990년에 귀국한 후 춘천의 한 폐교에 작업실을 마련하여 뉴욕과 한국을 오가며 작업했다. 작가는 1975년에 포드 재단 프랫 인스티튜트 스튜디오 미술상을 받고, 2002년 제14회 이중섭미술상, 2008년 제9회 이인성미술상 등을 수상했다. 김차섭은 1970년대까지 스크린 판화를 다수 제작했으나, 미국으로 건너간 후에는 에칭 기법을 활용한 판화작업을 했다. 김차섭의 에칭 판화는 기법뿐 아니라 작가가 그 당시 추구했던 기하학적 개념을 처음으로 시도한 것이었다.<직각>은 김차섭이 1976년 미국으로 가기 전부터 부단히 고민했던 ‘삼각형’에 대한 관념이 드러나 있다. 삼각형의 한 꼭짓점이 뚫려 있고 이를 가리키는 듯이 제스쳐를 한 손이 그려져 있으며, 열려 있는 직각 부분을 손 뒤의 예리한 각이 짚고 있다. 긴장감 있으면서 완벽한 도형으로 인식되는 삼각형의 직각 부분을 뚫은 것은 서구 합리주의에 대

In [20]:
# 특정 Document 객체의 텍스트 길이 확인
len(documents[11000].page_content)

970

In [21]:
# 800자를 초과하는 Document 개수 세기
over_800_count = sum(1 for doc in documents if len(doc.page_content) > 800)

# 결과 출력
print(f"800자를 초과하는 Document 개수: {over_800_count}")


800자를 초과하는 Document 개수: 4166


In [2]:
from langchain.vectorstores import FAISS
from sentence_transformers import SentenceTransformer
import faiss  # FAISS 라이브러리 필요

# 1. 임베딩 초기화
embedding_model = SentenceTransformer("nlpai-lab/KURE-v1")


In [22]:
# 2. 문서 데이터와 메타데이터 분리
texts = [doc.page_content for doc in chunked_documents]  # 문서 텍스트
metadatas = [doc.metadata for doc in chunked_documents]  # 문서 메타데이터

# 3. 문서 임베딩 생성
embeddings = embedding_model.encode(texts)

KeyboardInterrupt: 

In [4]:
# 4. FAISS 인덱스 생성
embedding_dim = embeddings.shape[1]  # 벡터 차원 확인
faiss_index = faiss.IndexFlatL2(embedding_dim)  # L2 거리 기반 인덱스
faiss_index.add(embeddings)  # 벡터 추가


NameError: name 'embeddings' is not defined

In [36]:
from langchain.docstore.in_memory import InMemoryDocstore

# 5. Docstore 생성
# 각 문서에 고유 ID를 부여해 InMemoryDocstore 생성
docstore = InMemoryDocstore({str(i): doc for i, doc in enumerate(chunked_documents)})

In [37]:
# 6. FAISS 벡터스토어 생성
def embed_query(text):
    return embedding_model.encode([text])[0]  # 단일 쿼리 텍스트를 임베딩

index_to_docstore_id = {i: str(i) for i in range(len(chunked_documents))}

faiss_db = FAISS(
    embedding_function=embed_query,
    index=faiss_index,
    docstore=docstore,
    index_to_docstore_id=index_to_docstore_id
)

# 7. FAISS 데이터베이스 저장
faiss_db.save_local("./faiss_artworks")
print("FAISS 데이터베이스가 성공적으로 저장되었습니다!")


`embedding_function` is expected to be an Embeddings object, support for passing in a function will soon be removed.


FAISS 데이터베이스가 성공적으로 저장되었습니다!


In [38]:
# 9. 검색 테스트
query = "이중섭 작품의 특징"
results = faiss_db.similarity_search(query, k=5)

# 10. 검색 결과 출력
for result in results:
    print("문서 텍스트:", result.page_content)
    print("문서 메타데이터:", result.metadata)


문서 텍스트: 작품 설명: 대향(大鄕) 이중섭(李仲燮, 1916-1956)은 평안북도 정주의 오산고등보통학교에서 서양화가 임용련, 백남순 부부에게 서양화를 배웠다. 이후 1936년 일본으로 건너가 데이코쿠미술학교(帝国美術学校)와 분카학원(文化学院)에서 미술을 전공했다. 추상 미술단체인 ‘자유미술가협회(自由美術家協会)’의 전시회에 지속적으로 출품하였으며, 제7회전(1943)에서는 태양상(太陽賞)을 수상했다. 1943년 귀국 후에는 생활고와 병으로 고생하면서도 꾸준히 작품을 제작했다.이중섭은 소, 아이들 등을 주요 소재로 고분 벽화와 민화 등 전통적이고 토속적인 것에 영감을 받아 표현주의적인 감각으로 작품을 제작했다. 이중섭의 작품에서는 그의 삶을 엿볼 수 있다. 동경의 분카학원에서 야마모토 마사코와 연애하던 시기의 엽서화에는 두 사람의 연인관계를 암시하는 환상적이고 초현실주의적인 이미지를 그렸다. 한국 전쟁기 제주도 피란시절 작품에는 가족과 행복했던 나날들이 소박하게 표현되었으며, 가족을 일본으로 보낸 후에는 삭막한 풍경화와 전쟁의 은유들이 그려졌다. 그는 열악한 경제 상황과 재료 부족에도 끊임없이 새로운 기법과 재료를 실험했는데, 담배를 싼 은지를 활용한 은지화가 대표적인 예이다. 전쟁이 끝난 후에는 가족을 만나려는 생각에 작품 제작에 몰두하여, 당당하고 힘찬 기세가 화면에 나타난다. 그러나 곧 경제적 어려움과 정신질환 등에 시달리며 가족과 재회할 수 있으리라는 희망이 사라졌을 때에는 초점을 잃은 흐릿한 풍경들이 애잔하게 펼쳐졌다.〈아이들〉은 수많은 아이들의 신체가 서로 포개어져 있거나 뒤엉켜 있는 모습을 담은 은지화이다. 신체의 구획을 명확히 헤아리기 어려울 정도로 복잡한 구성을 보인다. 아이들은 은근한 미소를 띠고 있으며, 역동적인 자세를 취하고 있는 이들은 마치 친구들과 장난을 치고 있는 듯 신나 보이기도 한다. 이중섭의 작품에서 아이들은 핵심적인 소재이다. 그러나 많은 경우 아이들은 물고기, 게 등 여러 동물과 혹은 식물과 어울리는 모습으로 자주 등장한

In [23]:
# 2. FAISS 데이터베이스 로드
persist_directory = "./faiss_artworks"

try:
    faiss_db = FAISS.load_local(
        folder_path=persist_directory,
        embeddings=embedding_model,
        allow_dangerous_deserialization=True  # 신뢰할 수 있는 소스에서만 사용
    )
    
    # embedding_function 수정
    faiss_db.embedding_function = lambda text: (
        embedding_model.encode(text) if isinstance(text, str) else embedding_model.encode(str(text))
    )
    
    print("FAISS 데이터베이스가 성공적으로 로드되었습니다!")
except Exception as e:
    print(f"FAISS 데이터베이스 로드 중 오류 발생: {e}")

`embedding_function` is expected to be an Embeddings object, support for passing in a function will soon be removed.


FAISS 데이터베이스가 성공적으로 로드되었습니다!


In [7]:
from transformers import AutoModelForCausalLM, AutoTokenizer, BitsAndBytesConfig
import torch

quantization_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype="float16",
    bnb_4bit_use_double_quant=True,
)

In [8]:
import torch
from langchain import HuggingFacePipeline
from transformers import AutoTokenizer, AutoModelForCausalLM, BitsAndBytesConfig, pipeline

# 모델과 토크나이저 로드 (CUDA 사용)
model_id = "LGAI-EXAONE/EXAONE-3.5-7.8B-Instruct"
tokenizer = AutoTokenizer.from_pretrained(model_id)
model = AutoModelForCausalLM.from_pretrained(
    model_id,
    quantization_config=quantization_config,
    device_map="cuda",  # CUDA에서 자동 배치
    trust_remote_code=True
)


Loading checkpoint shards: 100%|██████████| 7/7 [00:29<00:00,  4.20s/it]


In [9]:
from transformers import pipeline

# 파이프라인 생성
pipe = pipeline(
    "text-generation",
    model=model,
    tokenizer=tokenizer,
    max_new_tokens=1024,  # 생성할 최대 토큰 수 증가
    do_sample=True,        # 샘플링 활성화
    temperature=0.1,      
    top_k=50,             
    repetition_penalty=1.05
)
# LangChain의 HuggingFacePipeline 사용
llm = HuggingFacePipeline(pipeline=pipe)

Device set to use cuda
/tmp/ipykernel_1188/1315975019.py:15: LangChainDeprecationWarning: The class `HuggingFacePipeline` was deprecated in LangChain 0.0.37 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-huggingface package and should be used instead. To use it run `pip install -U :class:`~langchain-huggingface` and import as `from :class:`~langchain_huggingface import HuggingFacePipeline``.
  llm = HuggingFacePipeline(pipeline=pipe)


In [51]:
from langchain.prompts import ChatPromptTemplate

template = '''
<|system|>
You are a friendly chatbot specializing in artworks. 
Answer questions strictly based on the information provided in the document (context). 
If the requested information is not found in the document, respond with "The document does not contain this information." 
Provide detailed and comprehensive answers, always include the artwork number, and ensure all answers are written in Korean. 
All answers should be formatted using beautiful Markdown syntax to make the response visually appealing and easy to read. Use headings, bullet points, and bold or italic text where appropriate to enrich the response.

<|context|>
{context}

<|user|>
Question: {question}

<|assistant|>
'''


# 프롬프트 템플릿 생성
prompt = ChatPromptTemplate.from_template(template)


In [52]:
retriever = faiss_db.as_retriever(
    search_kwargs={
        "k": 5,                # 검색 결과 개수
        "fetch_k": 15,         # 더 많은 결과 가져오기
        "mmr": True,           # MMR 활성화
        "mmr_beta": 0.5        # 다양성과 관련성 간 균형
    }
)


In [53]:
import re
class MarkdownOutputParser:
    """Enhanced Markdown parser with additional formatting options."""

    def __call__(self, llm_output):
        # <assistant> 이후의 텍스트만 추출
        match = re.search(r"<\|assistant\|>\s*(.*)", llm_output, re.DOTALL)
        if match:
            extracted_text = match.group(1).strip()
            # 마크다운 코드 블록으로 출력 포맷
            return f"### 모델 결과\n\n{extracted_text}\n\n"
        else:
            # <assistant> 태그가 없는 경우 원래 출력 반환
            return f"### 모델 결과\n\n{llm_output.strip()}\n\n"


In [54]:
from langchain.schema.runnable import RunnablePassthrough, RunnableMap
from langchain_core.output_parsers.string import StrOutputParser
from langchain.prompts import ChatPromptTemplate
chain = (
    RunnableMap({
        "context": retriever,               # Retriever에서 반환된 값을 가져옴
        "question": RunnablePassthrough()   # 질문은 그대로 전달
    })
    | (lambda x: {
        "context": "\n".join([doc.page_content for doc in x["context"]]),
        "question": x["question"]
    })  # context를 문자열로 변환
    | prompt                               # Prompt Template에 전달
    | llm                                  # LLM으로 응답 생성
    | MarkdownOutputParser()                    # 응답을 문자열로 변환
)


In [57]:
query = "드로잉 작품 5개 추천해줘"

In [61]:
from IPython.display import Markdown, HTML, display

response = chain.invoke({"question": query})
display(Markdown(response))


In [48]:
type(retrieved_docs)

NameError: name 'retrieved_docs' is not defined

In [16]:
retrieved_docs = retriever.get_relevant_documents(query)
for i, doc in enumerate(retrieved_docs):
    print(f"Document {i+1}:")
    print(f"Content: {doc.page_content}")  # 문서의 실제 내용
    print(f"Metadata: {doc.metadata}")    # 메타데이터 (예: 출처, 페이지 등)
    print("-" * 50)


/tmp/ipykernel_1188/3695306690.py:1: LangChainDeprecationWarning: The method `BaseRetriever.get_relevant_documents` was deprecated in langchain-core 0.1.46 and will be removed in 1.0. Use :meth:`~invoke` instead.
  retrieved_docs = retriever.get_relevant_documents(query)


TypeError: string indices must be integers

In [ ]:
# 검색 수행: 유사도 점수와 함께 반환
docs_and_scores = retriever.vectorstore.similarity_search_with_score(query, k=5)

# 검색된 문서 수 출력
print(f"검색된 문서 수: {len(docs_and_scores)}")

# 각 문서의 파일명, 전체 내용, 유사도 점수 출력
for i, (doc, score) in enumerate(docs_and_scores, 1):
    print(f"\n문서 {i}:")
    print(f"  파일명: {doc.metadata.get('source', 'N/A')}")
    print(f"  유사도 점수: {score:.4f}")
    print(f"  전체 내용: {doc.page_content}")


검색된 문서 수: 5

문서 1:
  파일명: N/A
  유사도 점수: 0.6107
  전체 내용: 작품 설명: 이중섭(1916-1956)은 1916년 평안남도 평원의 부유한 가문에서 태어나, 정주 오산학교에서 임용련으로부터 미술지도를 받았고, 도쿄 제국미술학교와 문화학원에서 본격적으로 미술을 공부했다. 일제강점기 일본에서 화가 활동을 시작했고, 함경남도 원산으로 돌아온 후 해방을 맞았다. 한국전쟁으로 제주도, 부산 등지에서 피난생활을 했고, 전쟁 직후에는 통영, 서울, 대구 등지를 전전하며 열악한 환경 속에서도 열정적인 작품 활동을 하다가 1956년 만 40세의 나이로 생을 마감했다.이중섭은 1955년부터 극도의 좌절과 정신적인 압박에 시달리기 시작했고, 여러 병원을 전전하다가 1956년 서울의 정릉 골짜기에서 친구인 작가 한묵과 함께 지내고 있었다. 거식증으로 인한 영양실조, 그리고 간염 등으로 인해 매우 황폐한 생활을 하면서도, 일시적으로 상태가 좋아질 때는 여전히 끊임없이 작품을 제작했다.정릉시기 이중섭의 작품은 붉은 색을 포함한 강렬한 색을 거의 쓰지 않는다는 점, 흰 색과 우울한 노란색이 압도적이라는 점, 그리고 연필 위에 ‘크레파스’와 유채물감을 함께 섞어 여전히 기법적 실험을 계속하고 있다는 점을 들 수 있다. <정릉 풍경>(1956)은 1956년 9월 생을 마감하기 전 정릉에 머물렀던 짧은 기간 동안 제작된 작품으로 쓸쓸한 임종을 예견하고 있는 듯, 낮은 위치에서 골짜기의 경사를 올려다보는 불안한 시선을 택하고 있고, 여러 겹의 헝클어진 연필 선 위에 크레파스로 색을 쌓아 올리고, 그 위에 유채로 살짝 덧칠을 가하는 기법을 구사하고 있다. 쓸쓸하고 황량한 작가의 내면세계가 정릉의 흐릿한 풍경 속에 여지없이 녹아든 작품이라고 할 수 있다.

문서 2:
  파일명: N/A
  유사도 점수: 0.6621
  전체 내용: 작품 설명: 이중섭(1916-1956)은 1916년 평안남도 평원의 부유한 가문에서 태어나, 정주 오산학교에서 임용련으로부터 미술지

In [72]:
from langchain.vectorstores import Chroma
from langchain.embeddings import HuggingFaceEmbeddings

# 임베딩 모델 로드
embedding_model = HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2")

# Chroma 데이터베이스 불러오기 (임베딩 함수 포함)
persist_directory = './chroma_1104'
collection_name = 'chroma_art'

database = Chroma(
    embedding_function=embeddings,  # 임베딩 함수 설정
    collection_name=collection_name,
    persist_directory=persist_directory
)

# 모든 문서 검색을 위한 retriever 생성
retriever = database.as_retriever(search_kwargs={"k": 1000})

# 문서 검색 수행 (빈 쿼리 사용)
all_docs = retriever.get_relevant_documents(" ")

# 파일 소스 목록 출력
file_sources = set(doc.metadata['source'] for doc in all_docs)  # 중복 제거

print("데이터베이스에 저장된 파일 목록:")
for source in file_sources:
    print(source)


데이터베이스에 저장된 파일 목록:
./dataset2.docx
